In [ ]:
% matplotlib inline
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db = MySQLdb.connect(host="crunchbase.ckfkhzm7cot3.us-west-2.rds.amazonaws.com",
                     port=3306,
                     user="root",         # your username
                     passwd="root1234",  # your password
                     db="analytics_2")        # name of the data base


In [ ]:
cur = db.cursor()

In [ ]:
count = cur.execute("SELECT count(*) FROM cb_objects")

#### Load into pandas dataframe from database

In [ ]:
df_mysql = pd.read_sql('select * from cb_objects LIMIT 100;', con=db)

In [ ]:
persons = pd.read_sql('Select * from cb_objects where entity_type = \'Person\'', con=db)

In [ ]:
relationships = pd.read_sql('Select * from cb_relationships', con=db)

In [ ]:
degrees = pd.read_sql('Select * from cb_degrees', con=db)

In [ ]:
funding_rounds = pd.read_sql('Select * from cb_funding_rounds', con=db)

#### Fill nan for empty and None values

In [ ]:
persons.fillna(value=np.nan, inplace=True)
relationships.fillna(value=np.nan, inplace=True)
degrees.fillna(value=np.nan, inplace=True)

#### People

For the baseline model we need to consider only people who are Founders, CEO, CIO etc

#### Work Experience

In [ ]:
work_ex = pd.merge(persons, relationships, how='inner', right_on='person_object_id', left_on='id')

In [ ]:
print 'Total number of work ex rows', len(work_ex)

In [ ]:
workex_with_no_persons = work_ex[work_ex['id_x'].isnull()]
person_with_no_workex = work_ex[work_ex['id_y'].isnull()]
print "workex rows with no corresponding person",len(workex_with_no_persons)
print "Persons with no details about their workex",len(person_with_no_workex)

In [ ]:
print "Number of rows with no start_at for workex is: ",len(work_ex[work_ex['start_at'].isnull()]), "which is roughly half the number of rows"

Idea : We can fill the start_at date with one of the following:
1. founding_date of the company
2. created_at_y which is when the record was created in the database

To analyse we can first build a plot of the different between the 2 and see the relation between the title, created at and founding date. It can mean that most of the founders have their profile created on crunchbase much later than the founding date, in which case we can safely use the founding date. However, for employess we an use the created_at.

In [ ]:
work_ex['title'].unique

This below line of code filters all the work ex where the person is one of the founding members or the board of directors

In [ ]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Cheif|C[A-Z]O|Co\-Founder|Co\-founder|Board', regex=True),'id_x'])]

In [ ]:
# import dateutil
# founderspd.to_datetime(founders['start_at'])
# founders['start_at']
# len(founders[founders['start_at'].isnull()])

In [ ]:
founders.groupby('id_x').min()#agg({'start_at':min})

In [ ]:
founder_groups = founders.groupby('id_x')#.agg({'start_at':pd.datetime})
founder_groups.groups

In [ ]:
import datetime
# pd.to_datetime(founders['start_at'])[5] > pd.to_datetime('2012-01-01')
np.min(pd.to_datetime(founders['start_at']))

TODO: Aggregate as the min of teh start_at, this will give the first date when work ex added

#### Education

In [ ]:
len(degrees)

In [ ]:
degrees[:5]

In [ ]:
len(degrees['object_id'].unique())

We have degree details for the above number of people

We need to give a rating to the degrees. All Bachelors will have rating as 1. Masters will have rating 2 and PHD/MBA will have rating 3.As there are around 7000 differnet degrees, for other types of degrees we can use the median of the rest of the table. 

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('B\..*'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Bachelor'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('BS'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('JD'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Master'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('MS'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M.$'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M\..*'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Ph\.*D'),'degree_score'] = 3

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('MBA'),'degree_score'] = 3

In [ ]:
degrees.loc[degrees['degree_type'] == '','degree_score'] = 1

In [ ]:
degrees['degree_score'].fillna(np.median(degrees['degree_score']), inplace=True)

In [ ]:
degrees[['degree_type','degree_score']][:5]

In [ ]:
degrees.loc[degrees['degree_score'].isnull()]['degree_type'].describe()

In [ ]:
education = pd.merge(persons, degrees, how='inner', left_on='id', right_on='object_id')

In [ ]:
education_with_no_persons = education[education['id_x'].isnull()]
person_with_no_education = education[education['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Next Steps: 
1. We have sparse data for education. What to do with it?
2. How to clean workex data?

#### Funding Rounds

In [ ]:
len(funding_rounds)

In [ ]:
work_ex_funding = pd.merge(work_ex, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [ ]:
len(work_ex_funding)

In [ ]:
work_ex_funding[:5]
funding_rounds['funded_at_datetime']= pd.to_datetime(funding_rounds['funded_at'])
funding_rounds[['funded_at_datetime','funded_at']]

In [ ]:
kevin = work_ex_funding[work_ex_funding['id_x']=='p:3']
kevin[['title','funded_at']]
kevin

In [ ]:
kevin1 = work_ex[work_ex['id_x']=='p:3']
# kevin1[['title','funded_at']]
# kevin1['start_at_datetime'] = pd.to_datetime(kevin['start_at'].dropna(),yearfirst = True)
# pd.to_datetime(kevin['end_at'])
# pd.to_datetime(kevin['end_at'],infer_datetime_format=True)
# kevin1[['start_at', 'end_at','start_at_datetime', 'end_at_datetime']]
# kevin1[['start_at', 'end_at']]
# pd.to_datetime(kevin1['start_at'].dropna())

In [ ]:
a = '2001-02-01'
pd.to_datetime(a)

In [ ]:
a = work_ex[:5].groupby('id_x')
work_ex[:10]

### Train Test and Evaluation Data

We need to divide the data into train test and evaluation data. We will divide it in the following ratio:

1.Train - 60%

2.Test - 20%

3.Evaluation - 20%


### Baseline Model

Our baseline model is dependent on Education and work experience of the founder. We will train our linear regression model on the score of the founder based on the above 2 features and run it on the test data. 

We will then find the error rate of our baseline model on the test data.